In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master('local') \
        .appName('Quiz1') \
        .config('spark.ui.port', '4050') \
        .getOrCreate()

sc = spark.sparkContext

22/09/29 17:12:40 WARN Utils: Your hostname, orange resolves to a loopback address: 127.0.1.1; using 166.104.246.51 instead (on interface enp15s0)
22/09/29 17:12:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/29 17:12:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


문제 1.
* 5점을 받은 영화 중에서 알파벳 'T'로 시작하는 영화의 개수는?
* 'T'는 대문자
* 주의: 5점을 받은 영화중 알파벳 'T'로 시작하는 영화를 중복해서 세지 않도록!
* 제한조건: 마지막 action으로 답이 출력되도록 코드를 작성하시오. 즉, 최종 답을 계산하기 위해 pyspark action이 아닌 python 코드를 사용하지 마시오.

In [2]:
movies = sc.textFile('../../data/input3/u.item').map(lambda x: tuple(x.split('|')[:2]))
ratings = sc.textFile('../../data/input3/u.data').map(lambda x: tuple(x.split('\t'))[1:3])

In [3]:
start_with_t = movies.filter(lambda x: x[1][0] == 'T')
ratings_with_5 = ratings.filter(lambda x: x[1] == '5')
start_with_t.join(ratings_with_5).distinct().count()

75

문제 2.
* 사용자들에게 1점도 받고 5점도 받은 적이 있는 영화의 제목의 길이 중 가장 긴 영화제목의 길이는 [공백 1개]이다.
* 동일 영화가 어떤 사용자에게는 1점, 어떤 사용자에게는 5점을 받은 경우를 말함
* 힌트: cogroup을 쓸 수도 있고 groupByKey를 쓸 수도 있음.
* 제한조건: 마지막 action으로 답이 출력되도록 코드를 작성하시오. 즉, 최종 답을 계산하기 위해 pyspark action이 아닌 python 코드를 사용하지 마시오.

In [5]:
ratings_with_1 = ratings.filter(lambda x: x[1] == '1').map(lambda x: (x[0], '1')).distinct()
ratings_with_5 = ratings.filter(lambda x: x[1] == '5').map(lambda x: (x[0], '1')).distinct()
ratings_with_1_and_5 = ratings_with_1.intersection(ratings_with_5)
ratings_with_1_and_5.take(5)

[('346', '1'), ('796', '1'), ('328', '1'), ('145', '1'), ('96', '1')]

In [6]:
movies_with_1_and_5 = ratings_with_1_and_5.cogroup(movies).map(lambda x: (len(list(x[1][1])[0])))
movies_with_1_and_5.take(5)

[24, 47, 35, 13, 17]

In [7]:
movies_with_1_and_5.max()

81

In [8]:
sc.stop()